## In this file I will create the training loop for the NCE loss

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys


sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/")
sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/model")
from encoder_protein_dataset_nce import get_embedding, EncodedProteinDataset_nce#, collate_fn_new
from nce import nce, nce_old, loglik_potts, loglik_indep, sample_ind
import torch, torchvision
from torch.nn.utils import clip_grad_norm_
from potts_decoder_nce import PottsDecoder
from torch.utils.data import DataLoader, RandomSampler
from functools import partial
import biotite.structure
from biotite.structure.io import pdbx, pdb
from biotite.structure.residues import get_residues
from biotite.structure import filter_backbone
from biotite.structure import get_chains
from biotite.sequence import ProteinSequence
from typing import Sequence, Tuple, List
import scipy
from tqdm import tqdm
import pandas as pd
import csv
import time
from torch.utils.tensorboard import SummaryWriter

#import pytorch_warmup as warmup
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR

##TURIN HPC
#sys.path.insert(1, "/Data/silva/esm/")

## EUROPA
#sys.path.insert(1, "/home/lucasilva/esm/")

##Lucas computer
sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/esm/")
import esm
#from esm.inverse_folding import util
import esm.pretrained as pretrained
from ioutils import read_fasta, read_encodings
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict
from Bio import SeqIO

from dynamic_cluster_nce import dynamic_collate_fn, dynamic_cluster


In [2]:
### IDEA: MSAS PROCEDURE CAN GIVE DIFFERENT OUTPUT SHAPES? ASK
max_msas = None
#msa_dir = "/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/msas/"
msa_dir = "/home/luchinoprince/split2/"
encoding_dir ="/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/"

train_dataset = EncodedProteinDataset_nce(os.path.join(msa_dir, 'train'), encoding_dir, noise=0.02, max_msas=max_msas)          ## Default value of noise used
sequence_test_dataset = EncodedProteinDataset_nce(os.path.join(msa_dir, 'test/sequence'), encoding_dir, noise=0.0, max_msas=max_msas)
structure_test_dataset = EncodedProteinDataset_nce(os.path.join(msa_dir, 'test/structure'), encoding_dir, noise=0.0, max_msas=max_msas)
superfamily_test_dataset = EncodedProteinDataset_nce(os.path.join(msa_dir, 'test/superfamily'), encoding_dir, noise=0.0, max_msas=max_msas)


/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoder_protein_dataset_nce.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxh02.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/1vwxh02_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/3o58E00.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/3o58E00_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/2wc7A01.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/2wc7A01_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxP00.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/1vwxP00_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxG01.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/tr

In [3]:
print(f"I have loaded the train and test datasets: train:{len(train_dataset)}, seq:{len(sequence_test_dataset)}, struc:{len(structure_test_dataset)}, super:{len(superfamily_test_dataset)}")

I have loaded the train and test datasets: train:22463, seq:22396, struc:1364, super:2672


In [4]:
batch_structure_size = 32   ### I think with empty GPU we can go up to 16 easily
perc_subset_test = 1.0     ## During the training, for every dataset available we select a random 10% of its samples
batch_msa_size = 64 ### old is 32, original is 16
q = 21 ##isn't always 21
#dynamic_collate_fn

collate_fn = partial(dynamic_collate_fn, q=q, batch_size=batch_structure_size, batch_msa_size=batch_msa_size)
train_loader = DataLoader(train_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=True,
num_workers=4, pin_memory=True)


sequence_test_loader = DataLoader(sequence_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
num_workers=4, pin_memory=True, sampler=RandomSampler(sequence_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(sequence_test_dataset))))

structure_test_loader = DataLoader(structure_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
num_workers=4, pin_memory=True, sampler=RandomSampler(structure_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(structure_test_dataset))))

superfamily_test_loader = DataLoader(superfamily_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
num_workers=4, pin_memory=True, sampler=RandomSampler(superfamily_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(superfamily_test_dataset))))

In [5]:
decoder = None
embedding = None
torch.cuda.empty_cache()

seed = 24877
torch.random.manual_seed(seed)
np.random.seed(seed)



update_steps = 210000                                     ##Usual values are update steps=10^5, test_steps=10^2
test_steps = 7000
bk_iter = False                                                  ## This tells us how ofter we save a model(default values is every ten-thousand updates)
#n_epochs = update_steps//(len(train_dataset)//batch_structure_size)   ## the other update steps will be used for "partial epochs", I want to save the last complet epoch
#print(f"With update_steps:{update_steps} we will do {n_epochs} full epochs")

input_encoding_dim = 512
param_embed_dim = 512
n_param_heads = 48
d_model = 512 ##old 512
n_heads = 8 ## old 8
n_layers = 6
## Check before running which is the GPU which is free the most and put it as the running device
device = 0        ## DON'T SET TO ONE OTHER THAN IN SPECIAL SPECIAL OCCASIONS, VERY NOISYYYYY!
dropout = 0.1

decoder = PottsDecoder(q, n_layers, d_model, input_encoding_dim, param_embed_dim, n_heads, n_param_heads, dropout=dropout)
decoder.to(device)
embedding = get_embedding(q)
embedding.to(device)

512
512


Embedding(22, 21)

In [6]:
def get_loss_nce(decoder, inputs, mask_noise):
    ## For the moment no eta-J, maybe we need to add eventually also here
    msas, encodings, fi, padding_mask  = [input.to(device, non_blocking=True) for input in inputs]
    B, M, N = msas.shape

    
    #print(f"encodings' shape{encodings.shape}, padding mask:{padding_mask.shape}, msas: {msas.shape}, fi:{fi.shape}")
    param_embeddings, fields, logZ = decoder.forward(encodings, padding_mask)
    msas_embedded = embedding(msas)
    #eta = eta_J
    # get npll
    ## Get the samples from the independent modelpadding_mask
    samples_ind = torch.transpose(sample_ind(fi, n_samples=M), 0,1) 
    samples_ind = embedding(samples_ind)
    #print("Samples_indep", samples_ind)
    #print("check: ", torch.sum(samples_ind, dim=-1))
    ## We need to add the padding mask calculations!
    nce_loss = nce(param_embeddings, fields, logZ, padding_mask, msas_embedded, samples_ind, mask_noise, fi, N, M, q)
    #nce_loss = nce_old(param_embeddings, fields, logZ, msas_embedded, samples_ind, mask_noise, fi, N, M, q)
    #padding_mask_inv = (~padding_mask)
    return torch.mean(nce_loss)   ## We output a scalar

def get_loss_loader(decoder, loader, mask_noise):
    decoder.eval()
    losses = []
    #with torch.no_grad():
    for effective_batch_size, inputs_packed in loader:
        nce_full = 0
        for inputs in inputs_packed:
            mini_batch_size = inputs[0].shape[0]
            #_, npll = get_loss_indep(decoder, inputs, eta_J, eta_h) ## For independent model without couplings
            nce_loss = get_loss_nce(decoder, inputs, mask_noise[0:mini_batch_size, :])
            ## NCE here is (B, 1)
            nce_full += nce_loss*mini_batch_size/effective_batch_size
        losses.append(nce_full.item())
            #del inputs
    return np.mean(losses)

def train(decoder, inputs_packed, mask_noise, optimizer, scaler):
    effective_batch_size = inputs_packed[0]
    loss_penalty_full = 0
    #train_loss_full = 0
    optimizer.zero_grad(set_to_none=True)                           ## set previous gradients to 0
    with torch.cuda.amp.autocast():  ## autocasting mixed precision
        for inputs in inputs_packed[1]:
            mini_batch_size = inputs[0].shape[0]
            #loss_penalty, train_batch_loss = get_loss_indep(decoder, inputs, eta_J, eta_h)    ## get the current loss for the batch this is for the independent training
            nce_loss = get_loss_nce(decoder, inputs, mask_noise[0:mini_batch_size, :])
            nce_loss = nce_loss * mini_batch_size/effective_batch_size
            #print("NCE LOSS:", nce_loss)
            #print("ciao")
            #train_batch_loss = train_batch_loss * mini_batch_size/effective_batch_size
            #loss_penalty.backward()                         ## Get gradients
            scaler.scale(nce_loss).backward()
            loss_penalty_full += nce_loss.detach()
            #train_loss_full += nce
    
    
    scaler.step(optimizer)
    scaler.update()
    #optimizer.step()   

    return loss_penalty_full

In [7]:
#summary_writer = SummaryWriter(log_dir='runs/indep_model')#log_dir=logdir)
summary_writer = SummaryWriter(log_dir='./runs_nce/')
layout = {
    "metrics": {
        "loss": ["Multiline", ["loss/train", "loss/sequence", "loss/structure", "loss/superfamily"]],}
}
summary_writer.add_custom_scalars(layout)
lr = 1e-4
optimizer = torch.optim.AdamW(decoder.parameters(), lr=lr)



mask_noise = torch.zeros((batch_structure_size,2*batch_msa_size), requires_grad=False).to(device)
mask_noise[:, 0:batch_msa_size] = 1
#mask_noise = torch.zeros((1, 2*batch_msa_size), requires_grad=False).to(device)
#mask_noise[:, 0:batch_msa_size] = 1


start = time.time()
scaler = torch.cuda.amp.GradScaler()
with tqdm(total = update_steps) as pbar: ##This is used to have the nice loading bar while training
    train_loss = 0
    update_step = 0
    max_gpu=0
    train_batch_losses = []
    epoch = 0.0
    while update_step < update_steps:
        for inputs_packed in train_loader:
            decoder.train()

            loss_penalty = train(decoder, inputs_packed, mask_noise, optimizer, scaler)
            loss_penalty.detach()
            #optimizer.step()                                ## Do a step of GD
            update_step += 1                                ## Increase update step (the update steps will count also different batches within the same epoch)
            epoch = update_step / len(train_loader)
            
            train_batch_losses.append(loss_penalty.item()) ## Here we append the lossess in the different batches within the same epoch
            
            ## We want to keep track of the test loss not at every batch, too costrly otherwise. Usually set to once every 100.
            if (update_step == 1) or (epoch % 5 == 0):
                #check_aux = train_batch_losses.copy()
                #print(check_aux)
                train_loss = np.mean(train_batch_losses)
                with torch.no_grad():
                    #print("beginning first test")
                    sequence_test_loss = get_loss_loader(decoder, sequence_test_loader, mask_noise)
                    #print("Completed first test")
                    structure_test_loss = get_loss_loader(decoder, structure_test_loader, mask_noise)
                    #print("Completed second test")
                    superfamily_test_loss = get_loss_loader(decoder, superfamily_test_loader, mask_noise)
                    #print("Completed third test")

                summary_writer.add_scalar('loss/train', train_loss, update_step)
                summary_writer.add_scalar('loss/sequence', sequence_test_loss, update_step)
                summary_writer.add_scalar('loss/structure', structure_test_loss, update_step)
                summary_writer.add_scalar('loss/superfamily', superfamily_test_loss, update_step)

                ## UNCOMMENT THIS!
                train_batch_losses = []

        
            pbar.set_description(f'update_step: {update_step}, epoch: {epoch:.2f}  train: {train_loss:.2f}, sequence: {sequence_test_loss:.2f}, structure: {structure_test_loss:.2f}, superfamily: {superfamily_test_loss:.2f}')
            pbar.update(1)
            
print(f"It took {time.time()-start} seconds")
summary_writer.close()

  0%|          | 0/210000 [00:00<?, ?it/s]/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoder_protein_dataset_nce.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))
/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoder_protein_dataset_nce.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))
/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/util/encoder_protein_dataset_nce.py:85: UserWarning: To copy construct from 

It took 33794.916093587875 seconds


In [9]:
bk_dir= '/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/Intermediate_Models/Nce_Loss/'
fname_par = 'model_19_04_2023.pt'

##Arguments of the model, could be inferred
args = {}
args['n_layers'] = n_layers
args['input_encoding_dim'] = input_encoding_dim
args['param_embed_dim'] = param_embed_dim
args['n_heads'] = n_heads
args['n_param_heads'] = n_param_heads
args['dropout'] = dropout



d = {}
d['epoch'] = epoch
d['update_step'] = update_step
d['batch_size'] = batch_structure_size
d['seed'] = seed
#d['eta'] = eta
d['lr'] = lr 
d['noise'] = 0.02
d['args'] = args
d['model_state_dict'] = decoder.state_dict()
d['optimizer_state_dict'] = optimizer.state_dict()

torch.save(d, os.path.join(bk_dir, fname_par))

In [8]:
inputs_packed[1][0][0].shape

torch.Size([26, 32, 464])

In [9]:
mask_noise.shape

torch.Size([32, 64])

In [25]:
effective_batch_size = inputs_packed[0]
loss_penalty_full = 0
#train_loss_full = 0
optimizer.zero_grad(set_to_none=True)                           ## set previous gradients to 0
for inputs in inputs_packed[1]:
    msas, encodings, fi, padding_mask  = [input.to(device, non_blocking=True) for input in inputs]
    B, M, N = msas.shape
    print(B,M,N)
    #print(f"encodings' shape{encodings.shape}, padding mask:{padding_mask.shape}, msas: {msas.shape}, fi:{fi.shape}")
    param_embeddings, fields, logZ = decoder.forward(encodings, padding_mask)
    msas_embedded = embedding(msas)
    ## Get the samples from the independent modelpadding_mask
    samples_ind = torch.transpose(sample_ind(fi, n_samples=M), 0,1) 
    samples_ind = embedding(samples_ind)
    nce_loss = nce(param_embeddings, fields, logZ, padding_mask, msas_embedded, samples_ind, mask_noise, fi, N, M, q)


26 32 464


OutOfMemoryError: CUDA out of memory. Tried to allocate 62.00 MiB (GPU 0; 23.69 GiB total capacity; 23.09 GiB already allocated; 11.12 MiB free; 23.36 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [21]:
inputs_packed[1][0][0].shape

torch.Size([26, 32, 464])

In [149]:
llik_ind = loglik_indep(fi, msas_embedded, padding_mask)

lprobs: tensor([[[-0.0299, -0.0744, -0.0899,  ..., -0.4867, -0.2180, -0.0943],
         [-0.0299, -0.0744, -0.0899,  ..., -2.6471, -0.2180, -0.0943],
         [-0.0299, -0.0744, -0.0899,  ..., -2.9613, -2.6930, -0.0943],
         ...,
         [-0.0299, -0.0744, -0.0899,  ..., -0.4867, -0.2180, -0.0943],
         [-0.0299, -0.0744, -0.0899,  ..., -2.6471, -0.2180, -0.0943],
         [-0.0299, -0.0744, -4.4966,  ..., -0.4867, -0.2180, -0.0943]],

        [[-0.0637, -0.0770, -1.3499,  ...,    -inf,    -inf,    -inf],
         [-0.0637, -0.0770, -0.3340,  ...,    -inf,    -inf,    -inf],
         [-0.0637, -0.0770, -1.3499,  ...,    -inf,    -inf,    -inf],
         ...,
         [-0.0637, -0.0770, -1.3499,  ...,    -inf,    -inf,    -inf],
         [-0.0637, -0.0770, -1.3499,  ...,    -inf,    -inf,    -inf],
         [-0.0637, -0.0770, -0.3340,  ...,    -inf,    -inf,    -inf]]],
       device='cuda:0')
Fi:  tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.000

In [150]:
llik_ind

tensor([[-324.3916, -347.2759, -265.6017, -331.3303, -293.0596, -282.9628,
         -222.1092, -324.3026, -268.4892, -346.5865, -306.8795, -285.9696,
         -302.9697, -321.7809, -308.9503, -326.8862],
        [      nan,       nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan]], device='cuda:0')

In [156]:
lprobs=torch.log(torch.sum(torch.unsqueeze(fi, dim=1) * msas_embedded, axis=-1))
lprobs_i = lprobs * padding_mask_inv.unsqueeze(1)
lprobs_i = torch.nan_to_num(lprobs_i)

In [157]:
lprobs_i

tensor([[[-0.0299, -0.0744, -0.0899,  ..., -0.4867, -0.2180, -0.0943],
         [-0.0299, -0.0744, -0.0899,  ..., -2.6471, -0.2180, -0.0943],
         [-0.0299, -0.0744, -0.0899,  ..., -2.9613, -2.6930, -0.0943],
         ...,
         [-0.0299, -0.0744, -0.0899,  ..., -0.4867, -0.2180, -0.0943],
         [-0.0299, -0.0744, -0.0899,  ..., -2.6471, -0.2180, -0.0943],
         [-0.0299, -0.0744, -4.4966,  ..., -0.4867, -0.2180, -0.0943]],

        [[-0.0637, -0.0770, -1.3499,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0637, -0.0770, -0.3340,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0637, -0.0770, -1.3499,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.0637, -0.0770, -1.3499,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0637, -0.0770, -1.3499,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0637, -0.0770, -0.3340,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0')

In [143]:
padding_mask.shape

torch.Size([2, 154])

In [142]:
msas_embedded.shape

torch.Size([2, 16, 154, 21])

In [147]:
msas_embedded[1, 1, -1, :]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [138]:
lprobs[1, :, -1]

tensor([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       device='cuda:0')

In [119]:
padding_mask_inv = (~padding_mask)
lprobs_i2 = lprobs * padding_mask_inv.unsqueeze(1)

In [126]:
lprobs_i2.sum(axis=2)

tensor([[-280.4323, -306.9528, -279.8237, -272.4908, -297.5046, -312.1334,
         -284.2064, -313.7194, -293.6052, -297.2854, -271.8404, -317.1508,
         -306.3400, -301.3536, -300.0070, -297.8704],
        [-228.1575, -250.0851, -254.7909, -254.1803, -255.6519, -250.6403,
         -244.7155, -238.7070, -260.8159, -241.4896, -236.7061, -249.2051,
         -243.5480, -238.0499, -236.6011, -262.1979]], device='cuda:0')

In [106]:
samples_ind[1, :, -1]

tensor([17, 14,  9, 15,  8, 18, 13, 18, 11, 11, 15,  8, 19, 18, 20, 10],
       device='cuda:0')

In [80]:
samples_ind = torch.transpose(sample_ind(fi, n_samples=M), 0,1) 


In [116]:
lprobs[0,1,:]

tensor([0.9705, 0.9283, 0.9140, 0.8798, 0.0231, 0.8559, 0.8018, 0.8320, 0.8352,
        0.0581, 0.8272, 0.8185, 0.7317, 0.4514, 0.0581, 0.3917, 0.0621, 0.2500,
        0.4602, 0.0557, 0.1027, 0.1951, 0.2930, 0.1242, 0.0271, 0.1433, 0.2245,
        0.0191, 0.0573, 0.0470, 0.0796, 0.1457, 0.1688, 0.1385, 0.5717, 0.4459,
        0.0502, 0.6330, 0.8097, 0.1115, 0.0080, 0.2030, 0.0366, 0.2174, 0.1306,
        0.0438, 0.0199, 0.0143, 0.0358, 0.0438, 0.0764, 0.0382, 0.2182, 0.0454,
        0.0613, 0.0589, 0.2277, 0.1417, 0.0732, 0.0119, 0.0876, 0.3121, 0.2381,
        0.1568, 0.4729, 0.0717, 0.6346, 0.0900, 0.0565, 0.1990, 0.4029, 0.0167,
        0.8575, 0.5565, 0.0406, 0.2126, 0.0422, 0.2508, 0.7285, 0.0541, 0.2396,
        0.0565, 0.2635, 0.0932, 0.0549, 0.2659, 0.1186, 0.1003, 0.0422, 0.0932,
        0.1736, 0.0533, 0.1465, 0.0772, 0.0494, 0.1322, 0.1704, 0.1791, 0.0255,
        0.0939, 0.0438, 0.0963, 0.1688, 0.3209, 0.2962, 0.6194, 0.8033, 0.4968,
        0.8702, 0.2182, 0.0430, 0.2858, 

In [81]:
samples_ind[1, :, 1]

tensor([20, 20, 20, 20, 20,  5, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
       device='cuda:0')

In [91]:
fi[1,2,:]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0123, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7160, 0.0000, 0.0000,
        0.0000, 0.0123, 0.2593], device='cuda:0')

In [82]:
msas.shape

torch.Size([2, 16, 154])

In [28]:
msas[1, 1, -1]

tensor(21, device='cuda:0', dtype=torch.int32)

In [25]:
msas[1, 1, 1]

tensor(20, device='cuda:0', dtype=torch.int32)

In [29]:
msas_embedded[1, 1, -1, :]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [31]:
fi.shape

torch.Size([2, 154, 21])

In [36]:
torch.sum(fi[0, -2])

tensor(1.0000, device='cuda:0')

In [52]:
fi[1, -20, :]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.3951, 0.0000, 0.0000, 0.0617, 0.0000,
        0.0123, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0247,
        0.0123, 0.0494, 0.4444], device='cuda:0')

In [54]:
from torch.distributions import Categorical

In [56]:
?Categorical

Init signature: Categorical(probs=None, logits=None, validate_args=None)
Docstring:     
Creates a categorical distribution parameterized by either :attr:`probs` or
:attr:`logits` (but not both).

.. note::
    It is equivalent to the distribution that :func:`torch.multinomial`
    samples from.

Samples are integers from :math:`\{0, \ldots, K-1\}` where `K` is ``probs.size(-1)``.

If `probs` is 1-dimensional with length-`K`, each element is the relative probability
of sampling the class at that index.

If `probs` is N-dimensional, the first N-1 dimensions are treated as a batch of
relative probability vectors.

.. note:: The `probs` argument must be non-negative, finite and have a non-zero sum,
          and it will be normalized to sum to 1 along the last dimension. :attr:`probs`
          will return this normalized value.
          The `logits` argument will be interpreted as unnormalized log probabilities
          and can therefore be any real number. It will likewise be normaliz